# Deteción de riesgo de depresión/anxiety mediante ML (texto)

**Notebook listo para ejecutar en Google Colab.**

_Estructura:_ preprocesado, vectorización TF-IDF, modelos baseline (Logistic Regression), XGBoost, interpretabilidad (SHAP) y guardado de modelos.

**Nota:** Este notebook asume un CSV con columnas `text` y `label` (0/1). Ajusta rutas/columnas según el dataset que uses.

In [ ]:
# Instalación de dependencias (ejecutar en Colab)
!pip install -q scikit-learn xgboost shap kaggle nltk spacy joblib

# Descargar modelo spaCy en inglés (comenta si no lo necesitas)
!python -m spacy download en_core_web_sm

In [ ]:
# Imports principales
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, RocCurveDisplay, precision_recall_curve, auc
import xgboost as xgb
import joblib
import shap

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))  # cambiar a 'spanish' si es necesario


## Descarga del dataset desde Kaggle (opcional)

Sube tu `kaggle.json` a Colab y coloca en `~/.kaggle/kaggle.json`. Luego ejecuta la celda de abajo cambiando `<OWNER/DATASET>` por el identificador del dataset en Kaggle.

In [ ]:
# Ejemplo: descargar dataset desde Kaggle
# Asegúrate de subir tu kaggle.json al entorno de Colab antes de ejecutar
os.makedirs('/content/data/raw', exist_ok=True)
# !kaggle datasets download -d <OWNER>/<DATASET-NAME> -p /content/data/raw --unzip

# Si ya subiste un CSV manualmente, puedes copiarlo a /content/data/raw/data.csv
# from google.colab import files
# files.upload()

## Cargar datos

Asegúrate de que el CSV esté en `/content/data/raw/` y que tenga columnas `text` y `label`.

In [ ]:
# Cargar CSV (ajusta el nombre de archivo si procede)
csv_path = '/content/data/raw/data.csv'  # cambia si el archivo tiene otro nombre
if not os.path.exists(csv_path):
    print(f"Archivo {csv_path} no encontrado. Sube el dataset o ajusta la ruta.")
else:
    df = pd.read_csv(csv_path)
    print('Shape:', df.shape)
    display(df.head())

## Preprocesado de texto

In [ ]:
import re

def clean_text(s, lang='en'):
    s = str(s)
    s = re.sub(r'http\S+', ' ', s)
    s = re.sub(r'@\w+', ' ', s)
    s = re.sub(r'[^A-Za-zÀ-ÖØ-öø-ÿ0-9\s]', ' ', s)
    s = s.lower()
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# Aplicar limpieza si df existe
if 'df' in globals():
    if 'text' not in df.columns or 'label' not in df.columns:
        print('Asegúrate de que el dataset contenga columnas "text" y "label"')
    else:
        df['text_clean'] = df['text'].apply(lambda x: clean_text(x))
        df = df[df['text_clean'].str.len() > 10].reset_index(drop=True)
        print('After cleaning, shape:', df.shape)
        display(df.head())

## Exploración rápida

In [ ]:
if 'df' in globals():
    print(df['label'].value_counts())
    sns.countplot(x='label', data=df)
    plt.title('Distribución de etiquetas')
    plt.show()

## División train/test

In [ ]:
if 'df' in globals():
    X = df['text_clean'].values
    y = df['label'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        stratify=y, 
                                                        random_state=42)
    print('Train:', X_train.shape[0], 'Test:', X_test.shape[0])

## Baseline: TF-IDF + Logistic Regression

In [ ]:
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2), stop_words=STOPWORDS)

pipe_lr = Pipeline([
    ('tfidf', tfidf),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear'))
])

if 'X_train' in globals():
    pipe_lr.fit(X_train, y_train)
    y_pred = pipe_lr.predict(X_test)
    y_prob = pipe_lr.predict_proba(X_test)[:,1]
    print(classification_report(y_test, y_pred))
    print('ROC AUC:', roc_auc_score(y_test, y_prob))

## Modelo XGBoost (TF-IDF -> XGBoost)

In [ ]:
if 'X_train' in globals():
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    dtrain = xgb.DMatrix(X_train_tfidf, label=y_train)
    dtest = xgb.DMatrix(X_test_tfidf, label=y_test)

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'eta': 0.1,
        'max_depth': 6,
        'seed': 42
    }

    bst = xgb.train(params, dtrain, num_boost_round=200, evals=[(dtest,'test')], early_stopping_rounds=20)
    y_prob_xgb = bst.predict(dtest)
    y_pred_xgb = (y_prob_xgb >= 0.5).astype(int)

    print(classification_report(y_test, y_pred_xgb))
    print('ROC AUC XGBoost:', roc_auc_score(y_test, y_prob_xgb))

## Interpretabilidad: SHAP (muestra pequeña)

In [ ]:
if 'bst' in globals():
    # Para memoria, tomamos una muestra pequeña
    sample_idx = np.random.choice(X_test_tfidf.shape[0], size=min(200, X_test_tfidf.shape[0]), replace=False)
    X_shap = X_test_tfidf[sample_idx].toarray()
    dmat_shap = xgb.DMatrix(X_shap)
    explainer = shap.TreeExplainer(bst)
    shap_values = explainer.shap_values(dmat_shap)

    # Mostrar summary plot (puede tardar)
    try:
        feature_names = tfidf.get_feature_names_out()
    except:
        feature_names = [f'feat_{i}' for i in range(X_shap.shape[1])]
    shap.summary_plot(shap_values, X_shap, feature_names=feature_names[:X_shap.shape[1]], plot_type='bar')

## Guardar modelos y artefactos

In [ ]:
os.makedirs('/content/models', exist_ok=True)
if 'pipe_lr' in globals():
    joblib.dump(pipe_lr, '/content/models/pipe_lr.pkl')
if 'tfidf' in globals():
    joblib.dump(tfidf, '/content/models/tfidf.pkl')
if 'bst' in globals():
    bst.save_model('/content/models/xgb_model.json')
print('Modelos guardados en /content/models')

## Visualizaciones: ROC y matriz de confusión

In [ ]:
if 'y_prob_xgb' in globals():
    RocCurveDisplay.from_predictions(y_test, y_prob_xgb)
    plt.title('ROC Curve - XGBoost')
    plt.show()

if 'y_pred_xgb' in globals():
    cm = confusion_matrix(y_test, y_pred_xgb)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

## Próximos pasos y mejoras sugeridas

- Probar modelos basados en transformers (ClinicalBERT / DistilBERT) con `transformers` de Hugging Face.
- Añadir features demográficas o clínicos y combinarlas con texto (FeatureUnion).
- Evaluación clínica robusta: sensibilidad/especificidad en umbrales clínicos.
- Despliegue: crear una app con Streamlit para demo.
